## External imports

In [1]:
import os
import pandas as pd
import numpy as np

## Internal libraries imports

In [2]:
import wntr_WSN
import networkx_graph as ng
import linear_regression as lr

## Input data

In [3]:
method = 'method'                              # for file naming
leak_node = 'J64'                             # node that's leaking
comment = ''                                   # last word(s) in output filename
leak_start = 48                                # in hours
leak_end = 73                                  # in hours
leak_area = 0.0005
sim_duration = 130                             # total duration of the simulation

## Model each node with linear regression

In [4]:
linreg_models, normalization_params = lr.model_network_with_linreg(n=3)

Modeling the network with linreg (linear_regression.py - model_network_with_linregression())
Creating a NetworkX graph from .inp file (networkx_graph.py - create_graph())
Running the simulation (wntr_WSN.py - get_sim_results())


c:\users\pc\anaconda3\envs\wodociagi\lib\site-packages\scipy\optimize\minpack.py:828: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


	Modeling junction J1: (1 out of 309)
	Modeling junction J2: (2 out of 309)
	Modeling junction J3: (3 out of 309)
	Modeling junction J4: (4 out of 309)
	Modeling junction J5: (5 out of 309)
	Modeling junction J6: (6 out of 309)
	Modeling junction J7: (7 out of 309)
	Modeling junction J8: (8 out of 309)
	Modeling junction J9: (9 out of 309)
	Modeling junction J10: (10 out of 309)
	Modeling junction J11: (11 out of 309)
	Modeling junction J12: (12 out of 309)
	Modeling junction J13: (13 out of 309)
	Modeling junction J14: (14 out of 309)
	Modeling junction J15: (15 out of 309)
	Modeling junction J16: (16 out of 309)
	Modeling junction J17: (17 out of 309)
	Modeling junction J18: (18 out of 309)
	Modeling junction J19: (19 out of 309)
	Modeling junction J20: (20 out of 309)
	Modeling junction J21: (21 out of 309)
	Modeling junction J22: (22 out of 309)
	Modeling junction J24: (23 out of 309)
	Modeling junction J25: (24 out of 309)
	Modeling junction J26: (25 out of 309)
	Modeling junction

	Modeling junction J273: (251 out of 309)
	Modeling junction J274: (252 out of 309)
	Modeling junction J275: (253 out of 309)
	Modeling junction J276: (254 out of 309)
	Modeling junction J277: (255 out of 309)
	Modeling junction J278: (256 out of 309)
	Modeling junction J279: (257 out of 309)
	Modeling junction J280: (258 out of 309)
	Modeling junction J281: (259 out of 309)
	Modeling junction J282: (260 out of 309)
	Modeling junction J283: (261 out of 309)
	Modeling junction J284: (262 out of 309)
	Modeling junction J285: (263 out of 309)
	Modeling junction J286: (264 out of 309)
	Modeling junction J287: (265 out of 309)
	Modeling junction J288: (266 out of 309)
	Modeling junction J289: (267 out of 309)
	Modeling junction J290: (268 out of 309)
	Modeling junction J291: (269 out of 309)
	Modeling junction J292: (270 out of 309)
	Modeling junction J293: (271 out of 309)
	Modeling junction J294: (272 out of 309)
	Modeling junction J295: (273 out of 309)
	Modeling junction J296: (274 out 

In [5]:
linreg_models

[{'node': 'J1',
  'linreg': LinearRegression(),
  'msq': 0.0003094505413836043,
  'r2': 0.999779023346383,
  'sensors': ['J284', 'P9', 'P460']},
 {'node': 'J2',
  'linreg': LinearRegression(),
  'msq': 2.2718571746158296e-05,
  'r2': 0.9999860527644338,
  'sensors': ['J284', 'P460', 'P9']},
 {'node': 'J3',
  'linreg': LinearRegression(),
  'msq': 2.4049235727063265e-05,
  'r2': 0.9999835140029287,
  'sensors': ['J284', 'P460', 'P9']},
 {'node': 'J4',
  'linreg': LinearRegression(),
  'msq': 8.979415354764518e-05,
  'r2': 0.9999437589457921,
  'sensors': ['J284', 'P460', 'P9']},
 {'node': 'J5',
  'linreg': LinearRegression(),
  'msq': 6.921094605668927e-05,
  'r2': 0.9999575212346724,
  'sensors': ['J284', 'P460', 'P9']},
 {'node': 'J6',
  'linreg': LinearRegression(),
  'msq': 0.001315916827095419,
  'r2': 0.9993899974078964,
  'sensors': ['P9', 'J79', 'P500']},
 {'node': 'J7',
  'linreg': LinearRegression(),
  'msq': 0.00027063805115237587,
  'r2': 0.9998813887377228,
  'sensors': ['J

In [6]:
# simulate a leak scenario
sim_LEAK = wntr_WSN.get_sim_results_LEAK(leak_node, leak_area, leak_start, leak_end)

# join pressures and flowrates
sim_LEAK_raw = pd.concat([sim_LEAK.node['pressure'], sim_LEAK.link['flowrate']], axis=1)
predictions = {}

# for each node in graph
G = ng.create_graph()
for node in G.nodes():

    # find linear regression model for this particular node
    for model in linreg_models:
        if model['node'] == node:
            linreg_dict = model

    # choose only the available sensors
    sim_LEAK_scope = sim_LEAK_raw[linreg_dict['sensors']]

    # normalise the input data with respective parameters
    sim_LEAK_scope = (sim_LEAK_scope - normalization_params['mean'].loc[linreg_dict['sensors']]) / normalization_params['std'].loc[linreg_dict['sensors']]

    # predict pressure for each node
    predictions[node] = linreg_dict['linreg'].predict(sim_LEAK_scope)

# "sim_duration*6" because (60 minutes = 10 minutes * 6)
# it's related to simulation step time 
index = [600*i for i in range(0, (sim_duration*6)+1)]

pred_LEAK_results = pd.DataFrame(data=predictions, index=index)
pred_LEAK_results

c:\users\pc\anaconda3\envs\wodociagi\lib\site-packages\scipy\optimize\minpack.py:828: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


Creating a NetworkX graph from .inp file (networkx_graph.py - create_graph())


,J1,J2,J3,J4,J5,J6,J7,J8,J9,J10,...,J322,J323,J324,J326,J327,J539,J544,J546,T1,T2
0,36.007728,30.043946,29.924401,31.958709,32.148558,45.656096,31.842500,28.841451,33.860558,43.419479,...,39.770512,70.297167,43.725289,49.239213,27.804030,30.955127,83.333916,79.320369,25.459113,19.839908
600,38.995160,33.155931,33.036006,35.025148,35.215815,48.320933,34.824782,31.824736,36.845347,46.002609,...,42.109475,72.072027,45.880900,51.501378,30.138199,30.583525,85.060845,81.056026,28.321491,20.400044
1200,39.116052,33.277189,33.157258,35.146255,35.336938,48.442713,34.945636,31.945608,36.966216,46.124199,...,42.230851,72.193733,46.002615,51.622721,30.259576,30.704902,85.172661,81.168453,28.419035,20.520383
1800,36.748178,30.906448,30.786441,32.776993,32.967176,46.008250,32.578082,29.578119,34.598261,43.710317,...,39.060846,68.986034,43.241612,49.267862,27.100760,27.511339,24.344780,21.084591,26.512908,20.167304
2400,36.714150,30.872316,30.752312,32.742904,32.933083,45.973971,32.544065,29.544096,34.564239,43.676092,...,39.026682,68.951776,43.207352,49.233707,27.066595,27.477174,24.313286,21.052974,26.485451,20.133432
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
465600,32.274378,26.460614,26.340631,28.321744,28.511380,41.543831,28.104025,25.103830,30.124328,39.232900,...,34.504603,64.433670,38.671457,44.749682,22.543039,23.069615,20.160332,16.878780,23.059765,16.318970
466200,32.185867,26.373761,26.253760,28.234313,28.423923,41.455189,28.015443,25.015286,30.035751,39.144698,...,34.419044,64.348648,38.584743,44.663115,22.457443,22.984958,20.082207,16.800167,22.994623,16.240604
466800,32.100813,26.288083,26.168086,28.148874,28.338471,41.369494,27.930430,24.930259,29.950720,39.059392,...,34.334990,64.265235,38.500540,44.579043,22.373392,22.899159,20.005425,16.723079,22.924540,16.162398
467400,32.016400,26.203089,26.083097,28.064103,28.253688,41.284434,27.846055,24.845870,29.866329,38.974692,...,34.251417,64.182202,38.416806,44.495449,22.289821,22.814050,19.929131,16.646297,22.855141,16.084085


## Get results of leak scenario

In [7]:
sim_LEAK_results = pd.concat([sim_LEAK.node['pressure'], sim_LEAK.link['flowrate']], axis=1)
sim_LEAK_results

,J1,J2,J3,J4,J5,J6,J7,J8,J9,J10,...,P548,P549,P551,P553,PMP51,PMP63,PMP72,FCV51,FCV63,FCV72
0,36.011416,30.044852,29.924852,31.958688,32.148688,45.531102,31.841401,28.841400,33.861399,43.310270,...,0.005835,-0.001642,0.001183,-0.000037,0.020833,0.0,0.0434,0.020833,0.0,0.0434
600,38.996412,33.156409,33.036409,35.026409,35.216409,48.256448,34.826398,31.826397,36.846395,45.955310,...,-0.003068,0.000596,-0.000168,-0.000035,0.000000,0.0,0.0434,0.000000,0.0,0.0434
1200,39.117789,33.277785,33.157785,35.147786,35.337786,48.377824,34.947774,31.947773,36.967772,46.076686,...,-0.003068,0.000596,-0.000168,-0.000035,0.000000,0.0,0.0434,0.000000,0.0,0.0434
1800,36.745965,30.905961,30.785961,32.775962,32.965962,46.006000,32.575951,29.575950,34.595948,43.706562,...,0.001599,-0.000348,0.000072,-0.000008,0.000000,0.0,0.0000,0.000000,0.0,0.0000
2400,36.711800,30.871797,30.751797,32.741797,32.931797,45.971836,32.541786,29.541785,34.561783,43.672397,...,0.001599,-0.000348,0.000072,-0.000008,0.000000,0.0,0.0000,0.000000,0.0,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
465600,32.287546,26.463700,26.343700,28.328067,28.518067,41.516770,28.117472,25.117466,30.137446,39.212196,...,0.002238,-0.000472,-0.000261,-0.000033,0.000000,0.0,0.0000,0.000000,0.0,0.0000
466200,32.199657,26.376986,26.256986,28.240945,28.430945,41.429355,28.029243,25.029191,30.049191,39.124934,...,0.002277,-0.000481,-0.000258,-0.000033,0.000000,0.0,0.0000,0.000000,0.0,0.0000
466800,32.114172,26.291202,26.171202,28.155264,28.345264,41.344436,27.943758,24.943705,29.963705,39.040154,...,0.002303,-0.000487,-0.000255,-0.000033,0.000000,0.0,0.0000,0.000000,0.0,0.0000
467400,32.029338,26.206106,26.086106,28.070259,28.260259,41.260099,27.858923,24.858871,29.878871,38.955939,...,0.002325,-0.000493,-0.000252,-0.000033,0.000000,0.0,0.0000,0.000000,0.0,0.0000


## Calculate residuum

In [8]:
residuals = sim_LEAK.node['pressure'] - pred_LEAK_results
residuals

,J1,J10,J100,J102,J103,J104,J105,J106,J107,J108,...,J98,J99,JPMP51,JPMP63,JPMP72,R5,R6,R7,T1,T2
0,0.003687,-0.109209,0.011408,0.027907,0.013708,0.012505,-0.005332,-0.006691,-0.017601,0.035667,...,0.008589,0.027087,NaN,NaN,NaN,NaN,NaN,NaN,0.540887,0.160092
600,0.001252,-0.047299,-0.039084,-0.027194,-0.009116,0.003952,-0.004576,-0.005254,-0.007135,0.002183,...,-0.040772,-0.033877,NaN,NaN,NaN,NaN,NaN,NaN,-2.101202,-0.315379
1200,0.001736,-0.047513,-0.038637,-0.026882,-0.009232,0.003735,-0.004569,-0.005303,-0.007103,0.002171,...,-0.040328,-0.033429,NaN,NaN,NaN,NaN,NaN,NaN,-2.198747,-0.314342
1800,-0.002213,-0.003755,-0.003068,-0.002196,0.000417,0.002022,0.003481,0.002464,0.003436,0.001874,...,-0.003166,-0.001375,NaN,NaN,NaN,NaN,NaN,NaN,-0.292619,0.009757
2400,-0.002349,-0.003694,-0.003194,-0.002284,0.000450,0.002083,0.003479,0.002477,0.003427,0.001877,...,-0.003291,-0.001501,NaN,NaN,NaN,NaN,NaN,NaN,-0.265163,0.009464
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
465600,0.013168,-0.020704,-0.009884,-0.011965,-0.008830,-0.002837,-0.001542,-0.000557,-0.001875,-0.004576,...,-0.009853,-0.009901,NaN,NaN,NaN,NaN,NaN,NaN,-0.363185,-0.027191
466200,0.013791,-0.019764,-0.009016,-0.011202,-0.008611,-0.002678,-0.001509,-0.000502,-0.001896,-0.004208,...,-0.008908,-0.009230,NaN,NaN,NaN,NaN,NaN,NaN,-0.382425,-0.025979
466800,0.013359,-0.019238,-0.009300,-0.011429,-0.008618,-0.002499,-0.001507,-0.000471,-0.001955,-0.004175,...,-0.009180,-0.009518,NaN,NaN,NaN,NaN,NaN,NaN,-0.398718,-0.025284
467400,0.012937,-0.018753,-0.009584,-0.011652,-0.008616,-0.002324,-0.001506,-0.000441,-0.002010,-0.004145,...,-0.009452,-0.009806,NaN,NaN,NaN,NaN,NaN,NaN,-0.414936,-0.024740


## Take care of column naming convention

In [9]:
residuals = residuals.add_suffix('_residuum')

sim_LEAK.node['pressure'] = sim_LEAK.node['pressure'].add_suffix('_pressure')
sim_LEAK.link['flowrate'] = sim_LEAK.link['flowrate'].add_suffix('_flow')
sim_LEAK = pd.concat([sim_LEAK.node['pressure'], sim_LEAK.link['flowrate']], axis=1)

pred_LEAK_results = pred_LEAK_results.add_suffix('_model_output')

## Sanity check

In [10]:
print(f'Number of residuals columns: {len(residuals.columns)}')
print(f'Number of sim_LEAK columns: {len(sim_LEAK.columns)}')
print(f'Number of pred_LEAK_results columns: {len(pred_LEAK_results.columns)}')
print(f'Sum: {len(residuals.columns) + len(sim_LEAK.columns) + len(pred_LEAK_results.columns)}')

Number of residuals columns: 315
Number of sim_LEAK columns: 679
Number of pred_LEAK_results columns: 309
Sum: 1303


## Form final dataframe

In [11]:
output = pd.concat([residuals, sim_LEAK, pred_LEAK_results], axis=1)
output

,J1_residuum,J10_residuum,J100_residuum,J102_residuum,J103_residuum,J104_residuum,J105_residuum,J106_residuum,J107_residuum,J108_residuum,...,J322_model_output,J323_model_output,J324_model_output,J326_model_output,J327_model_output,J539_model_output,J544_model_output,J546_model_output,T1_model_output,T2_model_output
0,0.003687,-0.109209,0.011408,0.027907,0.013708,0.012505,-0.005332,-0.006691,-0.017601,0.035667,...,39.770512,70.297167,43.725289,49.239213,27.804030,30.955127,83.333916,79.320369,25.459113,19.839908
600,0.001252,-0.047299,-0.039084,-0.027194,-0.009116,0.003952,-0.004576,-0.005254,-0.007135,0.002183,...,42.109475,72.072027,45.880900,51.501378,30.138199,30.583525,85.060845,81.056026,28.321491,20.400044
1200,0.001736,-0.047513,-0.038637,-0.026882,-0.009232,0.003735,-0.004569,-0.005303,-0.007103,0.002171,...,42.230851,72.193733,46.002615,51.622721,30.259576,30.704902,85.172661,81.168453,28.419035,20.520383
1800,-0.002213,-0.003755,-0.003068,-0.002196,0.000417,0.002022,0.003481,0.002464,0.003436,0.001874,...,39.060846,68.986034,43.241612,49.267862,27.100760,27.511339,24.344780,21.084591,26.512908,20.167304
2400,-0.002349,-0.003694,-0.003194,-0.002284,0.000450,0.002083,0.003479,0.002477,0.003427,0.001877,...,39.026682,68.951776,43.207352,49.233707,27.066595,27.477174,24.313286,21.052974,26.485451,20.133432
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
465600,0.013168,-0.020704,-0.009884,-0.011965,-0.008830,-0.002837,-0.001542,-0.000557,-0.001875,-0.004576,...,34.504603,64.433670,38.671457,44.749682,22.543039,23.069615,20.160332,16.878780,23.059765,16.318970
466200,0.013791,-0.019764,-0.009016,-0.011202,-0.008611,-0.002678,-0.001509,-0.000502,-0.001896,-0.004208,...,34.419044,64.348648,38.584743,44.663115,22.457443,22.984958,20.082207,16.800167,22.994623,16.240604
466800,0.013359,-0.019238,-0.009300,-0.011429,-0.008618,-0.002499,-0.001507,-0.000471,-0.001955,-0.004175,...,34.334990,64.265235,38.500540,44.579043,22.373392,22.899159,20.005425,16.723079,22.924540,16.162398
467400,0.012937,-0.018753,-0.009584,-0.011652,-0.008616,-0.002324,-0.001506,-0.000441,-0.002010,-0.004145,...,34.251417,64.182202,38.416806,44.495449,22.289821,22.814050,19.929131,16.646297,22.855141,16.084085


## Export the final dataframe to .xlsx

In [12]:
if comment:
    output.to_excel(os.path.join(os.getcwd(), f'{method}_n{leak_node}_la{leak_area}_lst{leak_start}_let{leak_end}_{comment}.xlsx'))
else:
    output.to_excel(os.path.join(os.getcwd(), f'{method}_n{leak_node}_la{leak_area}_lst{leak_start}_let{leak_end}.xlsx'))